In [10]:
%matplotlib inline

from openquake.hazardlib.gsim import get_available_gsims, abrahamson_2018
from openquake.hazardlib.source import PointSource
from openquake.hazardlib.mfd import TruncatedGRMFD
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.geo import Point, NodalPlane, Line
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.site import Site, SiteCollection
from openquake.hazardlib.imt import PGA
from openquake.hazardlib.const import StdDev
from openquake.hazardlib.gsim.base import ContextMaker

import numpy

from matplotlib import pyplot, collections
from matplotlib.colorbar import cm
from collections import OrderedDict

In [2]:
# list available GMPEs
get_available_gsims().keys()

dict_keys(['AbrahamsonEtAl2014', 'AbrahamsonEtAl2014RegCHN', 'AbrahamsonEtAl2014RegJPN', 'AbrahamsonEtAl2014RegTWN', 'AbrahamsonEtAl2015SInter', 'AbrahamsonEtAl2015SInterHigh', 'AbrahamsonEtAl2015SInterLow', 'AbrahamsonEtAl2015SSlab', 'AbrahamsonEtAl2015SSlabHigh', 'AbrahamsonEtAl2015SSlabLow', 'AbrahamsonEtAl2018SInter', 'AbrahamsonEtAl2018SInterHigh', 'AbrahamsonEtAl2018SInterLow', 'AbrahamsonEtAl2018SSlab', 'AbrahamsonEtAl2018SSlabHigh', 'AbrahamsonEtAl2018SSlabLow', 'AbrahamsonSilva1997', 'AbrahamsonSilva2008', 'AfshariStewart2016', 'AfshariStewart2016Japan', 'AkkarBommer2010', 'AkkarBommer2010SWISS01', 'AkkarBommer2010SWISS04', 'AkkarBommer2010SWISS08', 'AkkarCagnan2010', 'AkkarEtAl2013', 'AkkarEtAlRepi2014', 'AkkarEtAlRhyp2014', 'AkkarEtAlRjb2014', 'AkkarEtAlRjb2014Armenia', 'AlAtikSigmaModel', 'Allen2012', 'AllenEtAl2012', 'AllenEtAl2012Rhypo', 'AmeriEtAl2017Repi', 'AmeriEtAl2017RepiStressDrop', 'AmeriEtAl2017Rjb', 'AmeriEtAl2017RjbStressDrop', 'Atkinson2008prime', 'Atkinson2010

In [4]:
for name, gmpe in get_available_gsims().items():
    print(name)
    print('supported tectonic region: %s'% gmpe.DEFINED_FOR_TECTONIC_REGION_TYPE)
    print('supported intensity measure types: %s' % ', '.join([imt.__name__ for imt in gmpe.DEFINED_FOR_INTENSITY_MEASURE_TYPES]))
    print('supported component: %s' % gmpe.DEFINED_FOR_INTENSITY_MEASURE_COMPONENT)
    print('supported standard deviations: %s' % ', '.join([std for std in gmpe.DEFINED_FOR_STANDARD_DEVIATION_TYPES]))
    print('required site parameters: %s' % ', '.join([p for p in gmpe.REQUIRES_SITES_PARAMETERS]))
    print('required rupture parameters: %s' % ', '.join([p for p in gmpe.REQUIRES_RUPTURE_PARAMETERS]))
    print('required distance parameters: %s' % ', '.join([p for p in gmpe.REQUIRES_DISTANCES]))

AbrahamsonEtAl2014
supported tectonic region: Active Shallow Crust
supported intensity measure types: PGA, SA, PGV
supported component: Average Horizontal (RotD50)
supported standard deviations: Intra event, Inter event, Total
required site parameters: z1pt0, vs30measured, vs30
required rupture parameters: ztor, rake, mag, width, dip
required distance parameters: ry0, rrup, rx, rjb
AbrahamsonEtAl2014RegCHN
supported tectonic region: Active Shallow Crust
supported intensity measure types: PGA, SA, PGV
supported component: Average Horizontal (RotD50)
supported standard deviations: Intra event, Inter event, Total
required site parameters: z1pt0, vs30measured, vs30
required rupture parameters: ztor, rake, mag, width, dip
required distance parameters: ry0, rrup, rx, rjb
AbrahamsonEtAl2014RegJPN
supported tectonic region: Active Shallow Crust
supported intensity measure types: PGA, SA, PGV
supported component: Average Horizontal (RotD50)
supported standard deviations: Intra event, Inter even

required distance parameters: rhypo
DouglasEtAl2013StochasticSD001Q200K005
supported tectonic region: Geothermal
supported intensity measure types: PGA, SA, PGV
supported component: Average horizontal
supported standard deviations: Intra event, Inter event, Total
required site parameters: 
required rupture parameters: mag
required distance parameters: rhypo
DouglasEtAl2013StochasticSD001Q200K020
supported tectonic region: Geothermal
supported intensity measure types: PGA, SA, PGV
supported component: Average horizontal
supported standard deviations: Intra event, Inter event, Total
required site parameters: 
required rupture parameters: mag
required distance parameters: rhypo
DouglasEtAl2013StochasticSD001Q200K040
supported tectonic region: Geothermal
supported intensity measure types: PGA, SA, PGV
supported component: Average horizontal
supported standard deviations: Intra event, Inter event, Total
required site parameters: 
required rupture parameters: mag
required distance parameters

TypeError: not enough arguments for format string

In [5]:
# select a number of GMPEs for which we want to analyze the magnitude scaling
from openquake.hazardlib.gsim.abrahamson_silva_2008 import AbrahamsonSilva2008
from openquake.hazardlib.gsim.chiou_youngs_2008 import ChiouYoungs2008
from openquake.hazardlib.gsim.campbell_bozorgnia_2008 import CampbellBozorgnia2008

gmpes = [AbrahamsonSilva2008(), ChiouYoungs2008(), CampbellBozorgnia2008()]

In [6]:
# explore magnitude scaling, by defining a Point source and calculating median ground shaking at the point
# source location
location = Point(9.1500, 45.1833)
src = PointSource(
    source_id='1',
    name='point',
    tectonic_region_type='Active Shallow Crust',
    mfd=TruncatedGRMFD(min_mag=5., max_mag=6.5, bin_width=0.1, a_val=0.01, b_val=0.98),
    rupture_mesh_spacing=2.,
    magnitude_scaling_relationship=WC1994(),
    rupture_aspect_ratio=1.,
    temporal_occurrence_model=PoissonTOM(50.),
    upper_seismogenic_depth=2.,
    lower_seismogenic_depth=12.,
    location=location,
    nodal_plane_distribution=PMF([(1., NodalPlane(strike=45, dip=50, rake=0))]),
    hypocenter_distribution=PMF([(1, 7.)])
)

# this is the site for which we compute the median ground shaking
site_collection = SiteCollection([Site(location=location, vs30=760., vs30measured=True, z1pt0=40., z2pt5=1.0)])

# this is the intensity measure type for which we compute the median ground shaking
imt = PGA()

In [24]:
# loop over ruptures. For each rupture extract magnitude and median value
means = []
mags = []
context_maker = ContextMaker(gsims=gmpes, trt=None)
for rupture in src.iter_ruptures():
    mags.append(rupture.mag)

    values = []
    for gmpe in gmpes:

        sctx, rctx, dctx = context_maker.make_contexts(site_collection, rupture)
        [mean], [std] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, PGA(), [StdDev.TOTAL])

        values.append(numpy.exp(mean))

    means.append(values)

mags = numpy.array(mags)
means = numpy.array(means).T

SyntaxError: positional argument follows keyword argument (<ipython-input-24-2ec840667b39>, line 12)

In [15]:
help(ContextMaker)

Help on class ContextMaker in module openquake.hazardlib.contexts:

class ContextMaker(builtins.object)
 |  A class to manage the creation of contexts for distances, sites, rupture.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, trt, gsims, param=None, monitor=<Monitor [gamant]>)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_rup_params(self, rupture)
 |      Add .REQUIRES_RUPTURE_PARAMETERS to the rupture
 |  
 |  filter(self, sites, rup)
 |      Filter the site collection with respect to the rupture.
 |      
 |      :param sites:
 |          Instance of :class:`openquake.hazardlib.site.SiteCollection`.
 |      :param rup:
 |          Instance of
 |          :class:`openquake.hazardlib.source.rupture.BaseRupture`
 |      :returns:
 |          (filtered sites, distance context)
 |  
 |  from_srcs(self, srcs, sites)
 |      :returns: a list of pairs (rctx, dctx)
 |  
 |  get_dctx(self, sites, rup)
 |      :param sites: :class:`openquake.haza